In [1]:
import collada as co
from shapely.geometry import Polygon
import geopandas as gpd
import pathlib
import xmltodict
import pandas as pd
import mio
import shapely

In [9]:
def dae2poly(model_path, x_shift, y_shift):
    """convert a collida model to shapely polygon"""
    model_path = str(model_path)
    mesh = co.Collada(model_path)
    # make polygonlist
    polys = []
    for geom in mesh.geometries:
        for triset in geom.primitives:
            for tri in triset:
                poly = Polygon(tri.vertices)
                polys.append(poly)
                
    gdf = gpd.GeoDataFrame(geometry=polys)
    gdf['id'] = 0
    gdf = gdf.dissolve('id')
    assert len(gdf), 'expect exactly one poy'
    poly = gdf.geometry[0]
    poly = shapely.affinity.translate(poly, xoff=x_shift, yoff=y_shift)
    return poly

In [10]:
def get_coords_df(kml_path):
    """read kml and return dataframe with coords and model name"""
    with open(kml_path) as fin:
        skml = fin.read()
    dic = xmltodict.parse(skml)
    pms = dic['kml']['Document']['Placemark']
    
    dics = []
    for pm in pms:
        model = pm['Model']
        lon = pm['Model']['Location']['longitude']
        lat = pm['Model']['Location']['latitude']
        model = pm['Model']['Link']['href']
        dic = {
            'lon' : lon, 
            'lat' : lat,
            'model' : model
        }
        dics.append(dic)
    #breakpoint()
    df = pd.DataFrame(dics)
    
    # add swiss coordinates
    coords = list(zip(df.lon, df.lat))
    coords = [mio.wgs_swiss(*ll) for ll in coords]
    coords = list(zip(*coords))
    df['x'] = coords[0]
    df['y'] = coords[1]
    return df

In [11]:
df = get_coords_df('swissBUILDINGS3D20_1166-42.kml')

In [15]:
geometry = []
for ind, row in df.iterrows():
    model = row['model']
    x, y = row['x'], row['y']
    poly = dae2poly(model, x, y)
    geometry.append(poly)

C:\tools\anaconda3\lib\site-packages\collada\util.py:70: RuntimeWarning: invalid value encountered in true_divide
  return vec / numpy.sqrt(numpy.vdot(vec, vec))


In [16]:
gdf = gpd.GeoDataFrame(geometry=geometry)

In [17]:
mio.write_tab(gdf, 'result.tab')

1503 row(s) written to mapinfo file.
